# 8. 错误和异常

## 8.3. 异常处理

In [1]:
while True:
    try:
        x = int(input('please input a integer: '))
        break
    except ValueError:
        print('Oops! That war not a integer, please try again.')

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

jupyter notebook无法使用交互式输入(input())，所以去在命令行中使用以上内容

一个 try 语句可能包含多个 except 子句，分别指定处理不同的异常。至多只会有一个分支被执行。异常处理程序只会处理对应的 try 子句中发生的异常，在同一个 try 语句中，其他子句中发生的异常则不作处理。一个 except 子句可以在括号中列出多个异常的名字

最后一个 except 子句可以省略异常名称，以作为通配符使用。你需要慎用此法，因为它会轻易隐藏一个实际的程序错误！可以使用这种方法打印一条错误信息，然后重新抛出异常（允许调用者处理这个异常):

例如:

In [8]:
import sys

try:
    f = open('./python3-tutorial/7.py')
    s = f.readline()
    i = int(s.strip())
except OSError as err:
    print("OS error: {0}".format(err))
except(RuntimeError, TypeError, NameError, ValueError):
    print("Could not convert data to an integer.")
except:
    print("Unexpected error:", sys.exc_info()[0])  # 打印一些信息
    raise  # 然后重新抛出这个异常

Could not convert data to an integer.


可以看到和Java等语言是类似的

前面在学习到Python的循环语句（while和for）时，看到它们可以带一个可选的else子句，这点与Java这种语言不一样。同样的，在Python中，try-except语句也可以带一个可选的else子句

try … except 语句可以带有一个 else子句，该子句只能出现在所有 except 子句之后。当 try 语句没有抛出异常时，需要执行一些代码，可以使用这个子句。例如:

In [11]:
for arg in sys.argv[1:]:
    try:
        f = open(arg, 'r')
    except IOError:
        print('cannot open', arg)
    else:   # 当try语句块没有抛出异常时，会执行else子句
        print(arg, 'has', len(f.readlines()), 'lines')
        f.close()

cannot open -f
C:\Users\heller\AppData\Roaming\jupyter\runtime\kernel-7d24dd07-8e98-4a18-b93d-8d3c31b5840f.json has 12 lines


使用 else 子句比在 try 子句中附加代码要好，因为这样可以避免 try … except 意外的截获本来不属于它们保护的那些代码抛出的异常。

异常捕获处理：
在异常名（列表）之后，也可以为 except 子句指定一个变量。这个变量绑定于一个异常实例，它存储在 instance.args 的参数中。为了方便起见，异常实例定义了 __str__() ，这样就可以直接访问过打印参数而不必引用 .args。这种做法不受鼓励。相反，更好的做法是给异常传递一个参数（如果要传递多个参数，可以传递一个元组），把它绑定到 message 属性。一旦异常发生，它会在抛出前绑定所有指定的属性。

In [21]:
try:
    raise Exception('spam', 'eggs')
except Exception as inst:   # as 
    print(type(inst))
    print(inst.args)    
    print(inst)  # __str__() allows args to be printed directly 类似Java中的toString()方法
                 # 同样的，这个方法也是可以被自定义的类重写的
    x, y = inst.args
    print('x=', x)
    print('y=', y)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')
x= spam
y= eggs


## 8.4. 抛出异常

raise 语句允许程序员强制抛出一个指定的异常。

要抛出的异常由 raise 的唯一参数标识。它必需是一个异常实例或异常类（继承自 Exception 的类）。

In [24]:
raise NameError('HiThere')

NameError: HiThere

如果你需要明确一个异常是否抛出，但不想处理它，raise 语句可以让你很简单的重新抛出该异常:

In [27]:
try:
    raise NameError('HiThere')
except NameError:
    print('An exception flew by!')
    raise # 重新抛出该异常

An exception flew by!


NameError: HiThere

## 8.5. 用户自定义异常
在程序中可以通过创建新的异常类型来命名自己的异常（Python 类的内容请参见 类 ）。异常类通常应该直接或间接的从 Exception 类派生，例如:（跟Java类似）

In [31]:
# 自定义一个异常，继承自Exception类
class MyError(Exception):   # 注意Python的自定义类和继承的语法
    def __init__(self, value): # 构造函数，self相当于Java中的this
        self.value = value
    
    def __str__(self):  # 类似Java中的toString()方法
        return repr(self.value)
    

# 试着用一下自定的异常类
try:
    raise MyError('Oops, error!')
except MyError as e:    # 看到了吧，跟Java是一样的
    print('MyError occurred, value: ', e.value) 

MyError occurred, value:  Oops, error!


在实际项目中 异常 使用的最佳实践：

如果一个新创建的模块中需要抛出几种不同的错误时，一个通常的作法是为该模块定义一个 异常基类 ，然后针对不同的错误类型派生出对应的 异常子类:

In [33]:
class Error(Exception):  # 定义模块中的异常 基类，继承自 Exception
    """Base class for exceptions in this module."""
    pass

class InputError(Error):  # 模块中的其他异常，继承自模块中定义的基类异常
    """Exception raised for errors in the input.

    Attributes:
        expression -- input expression in which the error occurred
        message -- explanation of the error
    """

    def __init__(self, expression, message):
        self.expression = expression
        self.message = message

class TransitionError(Error):   # 模块中的其他异常，继承自模块中定义的基类异常
    """Raised when an operation attempts a state transition that's not
    allowed.

    Attributes:
        previous -- state at beginning of transition
        next -- attempted new state
        message -- explanation of why the specific transition is not allowed
    """

    def __init__(self, previous, next, message):
        self.previous = previous
        self.next = next
        self.message = message

与标准异常相似，大多数异常的命名都以 “Error” 结尾。（Java中是Exception，注意这点不同）

## 8.6. 定义清理行为  （finally子句）
try 语句还有另一个可选的finally子句，目的在于定义在任何情况下都一定要执行的功能，跟Java一样
(finally子句块在try或者except语句块返回或再次抛出异常前执行)

In [38]:
try:
    raise KeyboardInterrupt
finally:
    print('Good bye, world!')

Good bye, world!


KeyboardInterrupt: 

上面可以清楚的看到，finally子句块在try子句块再次抛出异常前执行的

try 语句经由 break ，continue 或 return 语句退 出也一样会执行 finally 子句

不管有没有发生异常，finally子句 在程序离开 try 后都一定会被执行。当 try 语句中发生了未被 except 捕获的异常（或者它发生在 except 或 else 子句中），在 finally 子句执行完后它会被重新抛出。 try 语句经由 break ，continue 或 return 语句退 出也一样会执行 finally 子句。以下是一个更复杂些的例子:

In [43]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print('division by zero!')
    else:
        print('result is:', result)
    finally:
        print('executing finally clause')
        

divide(2, 1)

result is: 2.0
executing finally clause


In [44]:
divide(2, 0)

division by zero!
executing finally clause


In [45]:
divide('2', '1')

executing finally clause


TypeError: unsupported operand type(s) for /: 'str' and 'str'

可以清楚的看到，在各种情况下，finally子句块都执行了

如你所见， finally 子句在任何情况下都会执行。TypeError 在两个字符串相除的时候抛出，未被 except 子句捕获，因此在 finally 子句执行完毕后重新抛出。

在真实场景的应用程序中，finally 子句用于释放外部资源（文件 或网络连接之类的），无论它们的使用过程中是否出错。

跟Java完全一致

## 8.7. 预定义清理行为   (with子句)

有些对象定义了标准的清理行为，无论对象操作是否成功，不再需要该对象的时候就会起作用。以下示例尝试打开文件并把内容打印到屏幕上。

In [47]:
for line in open("./python3-tutorial/7.py"):
    print(line)

#!/usr/bin/env python3

# -*- coding: utf-8 -*-





这段代码的问题在于在代码执行完后没有立即关闭打开的文件。这在简单的脚本里没什么，但是大型应用程序就会出问题。with 语句使得文件之类的对象可以 确保总能及时准确地进行清理。

In [48]:
with open("./python3-tutorial/7.py") as f:
    for line in f:
        print(line)

#!/usr/bin/env python3

# -*- coding: utf-8 -*-





语句执行后，文件 f 总会被关闭，即使是在处理文件中的数据时出错也一样。其它对象是否提供了预定义的清理行为要查看它们的文档。

保证了资源使用后能够释放的需要

可以看到，这和Java 7 开始引入的 try-with-resource 特性是一致的

总结：
python的异常处理和Java基本完全类似，除了以下几个小点和语法上的细微区别
1. python的异常处理有else子句（python中的循环同样如此）	(严格来说，区别就这一条而已)

(2. with子句和Java 7后引入的try-with-resource类似，语法上细微区别而已)